In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=0.2,shear_range=0.2,
                             zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(r'C:\Users\KIIT\Desktop\Shubhangi\PROJECTS\TTL PROJECT\Prepared Data\train',
                                              target_size=(80,80),batch_size=8, class_mode='categorical', subset='training')

validation_data= train_datagen.flow_from_directory(r'C:\Users\KIIT\Desktop\Shubhangi\PROJECTS\TTL PROJECT\Prepared Data\train',
                                target_size=(80,80),batch_size=8,class_mode='categorical',subset='validation')

Found 61126 images belonging to 2 classes.
Found 15281 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\KIIT\Desktop\Shubhangi\PROJECTS\TTL PROJECT\Prepared Data\test',
                                              target_size=(80,80),batch_size=8, class_mode='categorical')



Found 8491 images belonging to 2 classes.


In [4]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 46s 1us/step


In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [8]:
checkpoint = ModelCheckpoint(r'C:\Users\KIIT\Desktop\Shubhangi\PROJECTS\TTL PROJECT\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [9]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
batchsize=8
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5


C:\Users\KIIT\AppData\Local\Temp\ipykernel_7600\2959890337.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


7640/7640 [==============================] - ETA: 0s - loss: 0.1572 - accuracy: 0.9398
Epoch 1: val_loss improved from inf to 0.25626, saving model to C:\Users\KIIT\Desktop\Shubhangi\PROJECTS\TTL PROJECT\models\model.h5
7640/7640 [==============================] - 565s 73ms/step - loss: 0.1572 - accuracy: 0.9398 - val_loss: 0.2563 - val_accuracy: 0.8970 - lr: 0.0010
Epoch 2/5
 607/7640 [=>............................] - ETA: 7:39 - loss: 0.1437 - accuracy: 0.9448

KeyboardInterrupt: 